## 1.

In [6]:
%reload_ext autoreload
%autoreload 2

In [7]:
from fastai.conv_learner import *
from fastai.models import darknet
from pathlib import Path

In [8]:
PATH = Path('data/imagenet')
PATH_TRAIN = PATH/'train'

In [5]:
# sz = 256
# bs = 32
# darknet53 = darknet.darknet_53()
# tfms = tfms_from_model(darknet53, sz)
# model_data = ImageClassifierData.from_paths(PATH, bs=bs, tfms=tfms, val_name='train')
# learner = ConvLearner.from_model_data(darknet53, model_data)

In [9]:
learner.crit

<function torch.nn.functional.nll_loss(input, target, weight=None, size_average=True, ignore_index=-100, reduce=True)>

In [12]:
model_data

In [13]:
learner.crit

<function torch.nn.functional.nll_loss(input, target, weight=None, size_average=True, ignore_index=-100, reduce=True)>

In [14]:
learner.crit = F.cross_entropy

In [15]:
learner.crit

<function torch.nn.functional.cross_entropy(input, target, weight=None, size_average=True, ignore_index=-100, reduce=True)>

## 2.

Checking behavior of `.pretrained` in fastai/conv_learner.py. Looking at number of FC layers at the end of a stock PyTorch model, compared to loading through FastAI:

In [9]:
from torchvision.models import resnet18

In [10]:
resnet18

<function torchvision.models.resnet.resnet18(pretrained=False, **kwargs)>

In [11]:
resnet18 = resnet18()

In [12]:
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (b

1 linear layer at the end.

In [13]:
from fastai.conv_learner import *

In [14]:
resnet18

<function torchvision.models.resnet.resnet18(pretrained=False, **kwargs)>

What fastai originally imports is the resnet18 constructor function from torchvision.models.resnet. Good to know.

In [15]:
from pathlib import Path
PATH = Path('../data/cifar10')

In [16]:
sz = 32
bs = 64
tfms = tfms_from_model(resnet18, sz)
model_data = ImageClassifierData.from_paths(PATH, bs, tfms=tfms, val_name='test')

In [17]:
learner = ConvLearner.pretrained(resnet18, model_data)

In [18]:
learner

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d

Right. fastai strips the FC layer and its associated pooling layers, and replaces them with adaptive pooling and 2 linear layers with batchNorm and dropout; with a LogSoftmax output layer.

In [25]:
from fastai.models import darknet

In [26]:
darknet.darknet_53

<function fastai.models.darknet.darknet_53(num_classes=1000)>

In [27]:
tfms = tfms_from_stats(imagenet_stats, sz)
model_data = ImageClassifierData.from_paths(PATH, bs=bs, tfms=tfms, val_name='test')

darknet53 = darknet.darknet_53

In [30]:
learner = ConvLearner.from_model_data(darknet53(num_classes=10), model_data)
learner

Darknet(
  (layers): Sequential(
    (0): ConvBN(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.01, affine=True)
      (relu): LeakyReLU(0.1, inplace)
    )
    (1): ConvBN(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.01, affine=True)
      (relu): LeakyReLU(0.1, inplace)
    )
    (2): DarknetBlock(
      (conv1): ConvBN(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.01, affine=True)
        (relu): LeakyReLU(0.1, inplace)
      )
      (conv2): ConvBN(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.01, affine=True)
        (relu): LeakyReLU(0.1, inplace)
      )
    )
    (3): ConvBN(
      (conv): Conv2d(64, 128, kernel_size=(3, 

In [29]:
learner = ConvLearner.pretrained(resnet18, model_data)
learner

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d